# Cómo crear embeddings con langchain

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

emb = embeddings.embed_query("hi")
emb

[-0.030890777707099915,
 -0.020455405116081238,
 -0.01946897804737091,
 -0.04176744818687439,
 -0.024972207844257355,
 0.02434920147061348,
 -0.01797635667026043,
 -0.01763889379799366,
 -0.006564290262758732,
 -0.016003498807549477,
 0.02594565786421299,
 -0.0070153214037418365,
 -0.017924439162015915,
 -0.011811180040240288,
 0.011408820748329163,
 0.01650969311594963,
 0.03888603672385216,
 0.0005836636992171407,
 0.032240625470876694,
 -0.008728591725230217,
 -0.019728563725948334,
 -0.004876978695392609,
 -0.009429474361240864,
 -0.01427725050598383,
 -0.022895516827702522,
 0.0025066309608519077,
 0.010039502754807472,
 -0.011791710741817951,
 0.0025715273804962635,
 -0.025971615687012672,
 0.01444598101079464,
 0.0007446883828379214,
 -0.03571908548474312,
 -0.014965154230594635,
 -0.0094489436596632,
 -0.024725601077079773,
 0.007002342492341995,
 -0.021221185103058815,
 0.0191185362637043,
 -0.0056654722429811954,
 0.006213848479092121,
 -0.0007657798123545945,
 0.001373374136

In [3]:
len(emb)

1536

# Qdrant

In [8]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [4]:
import os

url = "https://e7f4684c-fd33-4db0-b1d3-268870ecb84d.europe-west3-0.gcp.cloud.qdrant.io:6333"
api_key = os.getenv("QDRANT_API_KEY")

In [9]:
client = QdrantClient(
    url=url,
    api_key=api_key
)
client.create_collection(
    collection_name="dev-collection-test",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

True

In [11]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name="dev-collection-test",
    embedding=embeddings,
)

# Ingest Documents

In [12]:
import uuid
from langchain_core.documents import Document

In [13]:
vector_store.add_documents(
    documents=[Document(page_content="I love programming!", metadata={"theme": "software"})], 
    ids=[str(uuid.uuid4())])

['3ba0e35e-7916-4d96-b36f-71b8b0186f04']

# Make a search

In [15]:
vector_store.similarity_search(query="programming", k=2)

[Document(metadata={'theme': 'software', '_id': '3ba0e35e-7916-4d96-b36f-71b8b0186f04', '_collection_name': 'dev-collection-test'}, page_content='I love programming!')]

# Use filters

In [ ]:
from qdrant_client import models

results = vector_store.similarity_search(
    query="programming",
    k=1,
    filter=models.Filter(
        must=[
            models.FieldCondition(
                key="metadata.theme",
                match=models.MatchValue(
                    value="software"
                ),
            ),
        ]
    ),
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")